In [32]:
import numpy as np
from scipy import signal    # For signal.gaussian function
from copy import deepcopy
import cv2
import os

In [49]:
import cv2
import numpy as np
import os
datadir    = r'C:\Users\752 A BLK\Downloads\assignemnt2 (1)\assignemnt2\problem1\data'      # the directory containing the images
resultsdir = r'C:\Users\752 A BLK\Downloads\assignemnt2 (1)\assignemnt2\problem1\results6'

sigma     = 2
threshold = 0.03
rhoRes    = 2
thetaRes  = np.pi / 90
nLines    = 15

for file in os.listdir(datadir):
    if file.endswith('.jpg'):

        file = os.path.splitext(file)[0]
        
        # read in images
        img = cv2.imread('%s/%s.jpg' % (datadir, file))
        print(file)
        print(datadir)
        print(img)
        if (img.ndim == 3):
            img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        img = np.float32(img) / 255
        
        # actual Hough line code function calls
        img_edge = myEdgeFilter(img, sigma)
        img_threshold = np.float32(img_edge > threshold)
        [img_hough, rhoScale, thetaScale] = myHoughTransform(img_threshold, \
                                                             rhoRes, thetaRes)
        [rhos, thetas] = myHoughLines(img_hough, nLines)

        lines = cv2.HoughLinesP(np.uint8(255 * img_threshold), rhoRes, thetaRes, \
                                50, minLineLength = 20, maxLineGap = 5)

        # everything below here just saves the outputs to files
        fname = '%s/%s_01edge.png' % (resultsdir, file)
        cv2.imwrite(fname, 255 * np.sqrt(img_edge / img_edge.max()))
        
        fname = '%s/%s_02threshold.png' % (resultsdir, file)
        cv2.imwrite(fname, 255 * img_threshold)
        
        fname = '%s/%s_03hough.png' % (resultsdir, file)
        cv2.imwrite(fname, 255 * img_hough / img_hough.max())
        
        fname = '%s/%s_04lines.png' % (resultsdir, file)
        img_lines = np.dstack([img,img,img])

        # display line results from myHoughLines function in red
        for k in np.arange(nLines):
            a = np.cos(thetaScale[thetas[k]])
            b = np.sin(thetaScale[thetas[k]])
            
            x0 = a*rhoScale[rhos[k]]
            y0 = b*rhoScale[rhos[k]]
            
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            
            cv2.line(img_lines,(x1,y1),(x2,y2),(0,0,255),1)
        
        # display line segment results from cv2.HoughLinesP in green
        for line in lines:
            coords = line[0]
            cv2.line(img_lines, (coords[0], coords[1]), (coords[2], coords[3]), \
                     (0, 255, 0), 1)

        cv2.imwrite(fname, 255 * img_lines)



img01
C:\Users\752 A BLK\Downloads\assignemnt2 (1)\assignemnt2\problem1\data
[[[10 10 10]
  [35 35 35]
  [30 30 30]
  ...
  [17 17 17]
  [15 15 15]
  [16 16 16]]

 [[14 14 14]
  [37 37 37]
  [35 35 35]
  ...
  [17 17 17]
  [15 15 15]
  [16 16 16]]

 [[14 14 14]
  [34 34 34]
  [37 37 37]
  ...
  [17 17 17]
  [15 15 15]
  [16 16 16]]

 ...

 [[17 17 17]
  [23 23 23]
  [24 24 24]
  ...
  [17 17 17]
  [14 14 14]
  [17 17 17]]

 [[17 17 17]
  [22 22 22]
  [23 23 23]
  ...
  [17 17 17]
  [14 14 14]
  [17 17 17]]

 [[16 16 16]
  [22 22 22]
  [22 22 22]
  ...
  [17 17 17]
  [14 14 14]
  [17 17 17]]]
img02
C:\Users\752 A BLK\Downloads\assignemnt2 (1)\assignemnt2\problem1\data
[[[28 28 28]
  [28 28 28]
  [28 28 28]
  ...
  [16 16 16]
  [16 16 16]
  [15 15 15]]

 [[28 28 28]
  [28 28 28]
  [28 28 28]
  ...
  [16 16 16]
  [16 16 16]
  [15 15 15]]

 [[28 28 28]
  [28 28 28]
  [28 28 28]
  ...
  [16 16 16]
  [16 16 16]
  [15 15 15]]

 ...

 [[29 29 29]
  [29 29 29]
  [29 29 29]
  ...
  [16 16 16]
  

In [36]:
def clearArea(img, row, col):
    output_img = np.copy(img)
    threshold = 7
    for r in range(0, img.shape[0]):
        if abs(row - r) < threshold: 
            for c in range(0, img.shape[1]):
                if abs(col - c) < threshold: output_img[r,c] = 0
    return output_img

In [37]:
def kernel(img0,h,rowPsize, ColumnPsize, Imgpad, points):
    filteredImg = np.empty(np.shape(img0)) 
    filteredImg[:] = np.NaN
    for row in range(rowPsize, len(Imgpad)-rowPsize): 
        for col in range(ColumnPsize, len(Imgpad[0])-ColumnPsize):
            imgPoint = Imgpad[row-rowPsize:row+rowPsize+1, col-ColumnPsize:col+ColumnPsize+1]
            output = np.sum(np.multiply(imgPoint, h))/points
            filteredImg[row-rowPsize, col-ColumnPsize] = output
    for row in filteredImg:
        for elem in row:
            if np.isnan(elem):
                raise ValueError("nan value occured")        
    return filteredImg
def myImageFilter(img0, h):
    h_rows = np.shape(h)[0]
    h_cols = np.shape(h)[1]
    r_pad_size = int((h_rows - 1) / 2)
    c_pad_size = int((h_cols - 1) / 2)
    points = h_rows * h_cols
    Imgpad = np.pad(img0, ((r_pad_size,r_pad_size),(c_pad_size,c_pad_size)), 'edge')
    output = kernel(img0,h,r_pad_size,c_pad_size,Imgpad,points)
    return output

In [50]:
def DecideAngle(angle):
    if (22.5 <= angle and angle < 67.5): return 45
    if (67.5 <= angle and angle < 112.5): return 90
    if (112.5 <= angle and angle < 157.5): return 135
    else: return 0

In [41]:
def non_max_suppression(img, i, j, angle):
    if angle == 0: 
        if ((j-1 >= 0 and img[i, j-1] > img[i,j]) or
            (j+1 < np.shape(img)[1] and img[i, j+1] > img[i,j])):
            img[i,j] = 0
    elif angle == 135: 
        if ((i-1 >= 0 and j+1 < np.shape(img)[1] and img[i-1, j+1] > img[i,j]) or
            (i+1 < np.shape(img)[0] and j-1 >= 0 and img[i+1, j-1] > img[i,j])):
            img[i, j] = 0
    elif angle == 90: 
        if ((i-1 >= 0 and img[i-1, j] > img[i,j]) or
            (i+1 < np.shape(img)[0] and img[i+1, j] > img[i,j])):
            img[i,j] = 0
    elif angle == 45: 
        if ((i-1 >= 0 and j-1 >= 0 and img[i-1, j-1] > img[i,j]) or
            (i+1 < np.shape(img)[0] and j+1 < np.shape(img)[1] and img[i+1, j+1] > img[i,j])):
            img[i,j] = 0
    else:
        raise ValueError("magnitude direction calculating error")
    return img

In [51]:
def myEdgeFilter(img0, sigma):
    # smoothening img0
    hsize = 2 * math.ceil(3 * sigma) + 1
    gaussianFilter = np.reshape(signal.gaussian(hsize, sigma),(hsize,1))
    smoothImg = myImageFilter(img0, gaussianFilter)

    # Sobel Filter
    xSobel = [
        [1,0,-1],
        [2,0,-2],
        [1,0,-1]
    ] 
    ySobel = [
        [1,2,1],
        [0,0,0],
        [-1,-2,-1]
    ]

    imgX = myImageFilter(smoothImg,xSobel)
    imgY = myImageFilter(smoothImg,ySobel)
    imgXY = np.sqrt(imgX**2 + imgY**2)
    # non-maximum suppression
    thetas = (np.arctan2(imgY, imgX) * 180 / np.pi + 180 ) % 180
    for row in range(0, np.shape(thetas)[0]):
        for col in range(0, np.shape(thetas)[1]):
            gratitude_direction = DecideAngle(thetas[row,col])
            imgXY = non_max_suppression(imgXY, row, col, gratitude_direction)
    return imgXY

In [43]:
def myHoughLines(H, nLines):
    rhos = []
    thetas = [] 
    HCopy = np.copy(H)

    #find lines
    for i in range(0, nLines):
        location = np.unravel_index(np.argmax(HCopy, axis=None), HCopy.shape)
        rhos.append(location[0])
        thetas.append(location[1])
        HCopy = clearArea(HCopy, location[0], location[1])
    return [rhos, thetas]

In [44]:
def myHoughTransform(Im, rhoRes, thetaRes):
    Imgh = int(np.shape(Im)[0]) 
    Imgw = int(np.shape(Im)[1]) 
    M = np.ceil(np.sqrt(Imgh**2 + Imgw**2) + 1)
    rhoScale = np.arange(0,M,rhoRes)
    thetaScale = np.arange(0,np.pi*2,thetaRes)

    # we save votes here
    img_hough = np.zeros((np.shape(rhoScale)[0], np.shape(thetaScale)[0]), dtype=np.int32)
    for row in range(0,Imgh):
        for col in range(0, Imgw):
            if Im[row,col] > 0:
                for theta in thetaScale:
                    p = col * np.cos(theta) + row * np.sin(theta) 
                    if p >= 0:
                        rhoScale = np.asarray(rhoScale)
                        id_p = (np.abs(rhoScale - p)).argmin()
                        thetaScale = np.asarray(thetaScale)
                        id_theta = (np.abs(thetaScale - theta)).argmin()
                        img_hough[id_p, id_theta] +=1
    return [img_hough, rhoScale, thetaScale]